## **DATASET CREATION**

In [1]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import gensim
from gensim import corpora
from gensim.models.ldamodel import LdaModel
from nltk.corpus import stopwords
from langdetect import detect
import re
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from langdetect import detect
import spacy
from tqdm import tqdm
import pyLDAvis.gensim_models as gensimvis
import pickle 
import pyLDAvis
import os


/Users/filippofocaccia/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [8]:
politicians= pd.read_csv('politicians_data/politicians_classified.csv')

In [9]:
#drop the last column
politicians.drop(politicians.columns[-1], axis=1, inplace=True)


In [10]:
# Download NLTK resources
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

# Load stopwords
stop_words_italian = set(stopwords.words('italian'))
stop_words_english = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

# Load spaCy models for Italian and English
# You need to install these models first using:
nlp_it = spacy.load('it_core_news_sm')
nlp_en = spacy.load('en_core_web_sm')

def preprocess_text(text):
    try:
        # Detect language
        lang = detect(text)
    except:
        lang = 'unknown'
    
    # Remove URLs, mentions, hashtags, and punctuation
    text = re.sub(r"http\S+|www\S+|https\S+", '', text, flags=re.MULTILINE)
    text = re.sub(r'\@\w+|\#', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    
    if lang == 'it':  # Italian
        # Process with spaCy for Italian
        doc = nlp_it(text.lower())
        # Get lemmas and filter stopwords, short words, verbs, and adverbs
        true_tokens = [token.lemma_ for token in doc 
                      if token.text not in stop_words_italian and len(token.text) > 2 and token.pos_ not in ('VERB', 'ADV','AUX')]
    
    elif lang == 'en':  # English
        # Process with spaCy for English
        doc = nlp_en(text.lower())
        # Get lemmas and filter stopwords, short words, verbs, and adverbs
        true_tokens = [token.lemma_ for token in doc 
                      if token.text not in stop_words_english and len(token.text) > 2 and token.pos_ not in ('VERB', 'ADV')]
    
    else: 
        # Fallback to NLTK for spacy errors or mislablings
        tokens = word_tokenize(text.lower())
        true_tokens = [lemmatizer.lemmatize(word) for word in tokens if len(word) > 2]
    
    return true_tokens

# Apply preprocessing to the tweets
politicians_tweets = politicians['Content']

# Method 1: Using tqdm with apply (simpler)
tqdm.pandas(desc="Processing tweets")
politicians['processed_tweet'] = politicians_tweets.progress_apply(lambda x: preprocess_text(x))


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/filippofocaccia/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/filippofocaccia/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/filippofocaccia/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
Processing tweets: 100%|██████████| 17245/17245 [04:39<00:00, 61.75it/s] 


### Let's perform topic modeling on all the politicians' tweets using LDA, initially we perform it on the whole dataset

In [11]:
dictionary = corpora.Dictionary(politicians['processed_tweet'])
corpus = [dictionary.doc2bow(text) for text in politicians['processed_tweet']]


In [12]:
num_topics = 3  # Numero di topic da identificare

In [13]:
lda_model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=num_topics, passes=10, random_state=42)

In [14]:
for idx, topic in lda_model.print_topics(-1):
    print(f"Topic {idx}: {topic}")

Topic 0: 0.016*"governo" + 0.013*"lavoro" + 0.008*"drago" + 0.007*"lavoratore" + 0.007*"proposta" + 0.006*"scuola" + 0.006*"impresa" + 0.005*"famiglia" + 0.005*"anno" + 0.005*"vaccino"
Topic 1: 0.009*"paese" + 0.008*"politico" + 0.008*"Roma" + 0.007*"politica" + 0.007*"grande" + 0.006*"anno" + 0.005*"italiano" + 0.005*"cittadino" + 0.005*"ogni" + 0.005*"libertà"
Topic 2: 0.011*"destra" + 0.010*"cosa" + 0.009*"governo" + 0.008*"salvino" + 0.006*"sinistra" + 0.006*"italiano" + 0.006*"conte" + 0.006*"parola" + 0.006*"drago" + 0.005*"ministro"


In [15]:
pyLDAvis.enable_notebook()
LDAvis_data_filepath = os.path.join('./results/ldavis_prepared_total'+str(num_topics))

if 1 == 1:
    LDAvis_prepared = gensimvis.prepare(lda_model, corpus, dictionary)
    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)

with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)

pyLDAvis.save_html(LDAvis_prepared, './results/ldavis_prepared_total'+ str(num_topics) +'.html')

LDAvis_prepared

/Users/filippofocaccia/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
/Users/filippofocaccia/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
/Users/filippofocaccia/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
/Users/filippofocaccia/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
/Users/filippofocaccia/anaconda3/lib/python3.11/site-packages/pandas/cor

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.038606 -0.172257       1        1  42.315837
0     -0.153277  0.115029       2        1  34.525950
2      0.191883  0.057228       3        1  23.158213, topic_info=            Term         Freq        Total Category  logprob  loglift
1248      destra   539.000000   539.000000  Default  30.0000  30.0000
298       lavoro  1286.000000  1286.000000  Default  29.0000  29.0000
3037     salvino   422.000000   422.000000  Default  28.0000  28.0000
540   lavoratore   559.000000   559.000000  Default  27.0000  27.0000
22          Roma   698.000000   698.000000  Default  26.0000  26.0000
...          ...          ...          ...      ...      ...      ...
802     politico   157.407130   903.104167   Topic3  -5.7688  -0.2842
646        altro   151.328442   902.435352   Topic3  -5.8082  -0.3228
232        volta   131.238009   385.887754   Topic3  -5.9506   0.3843
444     politica   141.184619   942.376873   Topic3  -5.8776  -0.4355
90        giorno   137.154215   687.353326   Topic3  -5.9065  -0.1489

[221 rows x 6 columns], token_table=      Topic      Freq    Term
term                         
199       1  0.491225  Italia
199       2  0.229046  Italia
199       3  0.279465  Italia
2640      1  0.993154  Milano
22        1  0.997068    Roma
...     ...       ...     ...
232       2  0.204723   volta
232       3  0.339477   volta
811       1  0.396223    voto
811       2  0.137513    voto
811       3  0.466144    voto

[322 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 1, 3])

### Then, we divide the politicians in separate datasets with respect to their belief

In [16]:
right_wing=politicians[politicians['ideology_num'] == 3]
left_wing=politicians[politicians['ideology_num'] == 0]

### Let's perform topic modeling now on right wing politicians

In [17]:
dictionary_right = corpora.Dictionary(right_wing['processed_tweet'])
corpus_right = [dictionary_right.doc2bow(text) for text in right_wing['processed_tweet']]
# 4. Applicazione di LDA
lda_model_right= LdaModel(corpus=corpus_right, id2word=dictionary_right, num_topics=num_topics, passes=10, random_state=42)
# 6. Stampa dei topic
for idx, topic in lda_model_right.print_topics(-1):
    print(f"Topic {idx}: {topic}")



Topic 0: 0.009*"anno" + 0.008*"lega" + 0.008*"vita" + 0.007*"solidarietà" + 0.006*"famiglia" + 0.006*"donna" + 0.005*"tutto" + 0.005*"grazie" + 0.005*"italiano" + 0.005*"scuola"
Topic 1: 0.017*"italiano" + 0.011*"sinistra" + 0.009*"ministro" + 0.006*"paese" + 0.005*"altro" + 0.005*"settembre" + 0.005*"litalia" + 0.005*"libertà" + 0.005*"Italia" + 0.005*"clandestino"
Topic 2: 0.019*"governo" + 0.012*"lavoro" + 0.010*"italiano" + 0.008*"impresa" + 0.008*"paese" + 0.007*"presidente" + 0.007*"lega" + 0.006*"centrodestra" + 0.006*"drago" + 0.006*"famiglia"


In [18]:
pyLDAvis.enable_notebook()
LDAvis_data_filepath = os.path.join('./results/ldavis_prepared_right'+str(num_topics))

if 1 == 1:
    LDAvis_prepared_right = gensimvis.prepare(lda_model_right, corpus_right, dictionary_right)
    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)

with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)

pyLDAvis.save_html(LDAvis_prepared_right, './results/ldavis_prepared_right'+ str(num_topics) +'.html')

LDAvis_prepared_right

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.156423  0.071567       1        1  43.849590
1     -0.004721 -0.150030       2        1  29.163041
0     -0.151703  0.078463       3        1  26.987369, topic_info=             Term        Freq       Total Category  logprob  loglift
26        governo  992.000000  992.000000  Default  30.0000  30.0000
237       impresa  354.000000  354.000000  Default  29.0000  29.0000
376    presidente  332.000000  332.000000  Default  28.0000  28.0000
962      sinistra  454.000000  454.000000  Default  27.0000  27.0000
924  centrodestra  292.000000  292.000000  Default  26.0000  26.0000
..            ...         ...         ...      ...      ...      ...
298        lavoro  110.189205  694.080608   Topic3  -5.5388  -0.5306
530        parola   82.885867  225.887054   Topic3  -5.8235   0.3072
242          dopo   86.419523  291.474923   Topic3  -5.7817   0.0941
22           Roma   79.044383  242.246802   Topic3  -5.8709   0.1899
462       sindaco   74.615775  184.179430   Topic3  -5.9286   0.4062

[226 rows x 6 columns], token_table=      Topic      Freq                 Term
term                                      
9850      1  0.008036  25settembrevotolega
9850      2  0.980373  25settembrevotolega
9850      3  0.008036  25settembrevotolega
199       1  0.541040               Italia
199       2  0.319191               Italia
...     ...       ...                  ...
345       3  0.668867                 vita
637       2  0.008941              vittima
637       3  0.983530              vittima
2119      2  0.104391                 zona
2119      3  0.887326                 zona

[338 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 2, 1])

### Topics:
- Topic 1: "Traditional Family Values": 
    This topic centers around family, women, solidarity, life, and education. The prominence of words like "famiglia" (family), "donna" (woman), "vita" (life), and "solidarietà" (solidarity) suggests a focus on traditional social values and family structures. The appearance of "scuola" (school) indicates emphasis on education within this traditional values framework.


- Topic 2: "National Identity and Immigration"
    This topic features words related to national identity and immigration concerns. Key terms include "italiano" (Italian), "sinistra" (left/left-wing), "paese" (country), "libertà" (freedom), "Italia" (Italy), and importantly "clandestino" (illegal immigrant). This suggests rhetoric focused on opposition to immigration, criticism of left-wing policies, and emphasis on national sovereignty and Italian identity.


- Topic 3: "Economic Governance and Center-Right Coalition":
    This topic revolves around governance and economic issues. Prominent words include "governo" (government), "lavoro" (work), "impresa" (business/enterprise), "presidente" (president), "centrodestra" (center-right), and "drago" (possibly referring to Mario Draghi). This suggests rhetoric focused on economic policy, business interests, and center-right governance.

### Let's now proceed with left wing politicians

In [19]:
dictionary_left = corpora.Dictionary(left_wing['processed_tweet'])
corpus_left = [dictionary_left.doc2bow(text) for text in left_wing['processed_tweet']]
lda_model_left = LdaModel(corpus=corpus_left, id2word=dictionary_left, num_topics=num_topics, passes=10, random_state=42)
# 6. Stampa dei topic
for idx, topic in lda_model_left.print_topics(-1):
    print(f"Topic {idx}: {topic}")


Topic 0: 0.009*"politica" + 0.009*"cosa" + 0.009*"governo" + 0.008*"destra" + 0.006*"Roma" + 0.006*"politico" + 0.006*"voto" + 0.006*"partito" + 0.005*"persona" + 0.005*"serio"
Topic 1: 0.008*"diritto" + 0.006*"anno" + 0.006*"libertà" + 0.005*"lavoro" + 0.005*"donna" + 0.005*"persona" + 0.005*"altro" + 0.005*"senza" + 0.005*"tutto" + 0.004*"italiano"
Topic 2: 0.015*"governo" + 0.009*"lavoro" + 0.008*"drago" + 0.007*"paese" + 0.007*"lavoratore" + 0.005*"crisi" + 0.005*"proposta" + 0.004*"legge" + 0.004*"altro" + 0.004*"parlamento"


In [20]:
pyLDAvis.enable_notebook()
LDAvis_data_filepath = os.path.join('./results/ldavis_prepared_left'+str(num_topics))

if 1 == 1:
    LDAvis_prepared_left = gensimvis.prepare(lda_model_left, corpus_left, dictionary_left)
    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)

with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)

pyLDAvis.save_html(LDAvis_prepared_left, './results/ldavis_prepared_left'+ str(num_topics) +'.html')

LDAvis_prepared_left

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.010307 -0.158743       1        1  36.103756
1     -0.134865  0.088136       2        1  32.017969
0      0.145172  0.070607       3        1  31.878275, topic_info=            Term        Freq       Total Category  logprob  loglift
1857  lavoratore  309.000000  309.000000  Default  30.0000  30.0000
138      libertà  217.000000  217.000000  Default  29.0000  29.0000
46         donna  189.000000  189.000000  Default  28.0000  28.0000
45       diritto  432.000000  432.000000  Default  27.0000  27.0000
189    candidato  187.000000  187.000000  Default  26.0000  26.0000
...          ...         ...         ...      ...      ...      ...
156        parte  127.567564  364.406230   Topic3  -5.6499   0.0936
2           anno  131.672640  512.431373   Topic3  -5.6182  -0.2156
149     italiano  129.757772  444.523086   Topic3  -5.6329  -0.0881
5           dopo  126.867895  363.104183   Topic3  -5.6554   0.0917
179        altro  127.135239  477.291193   Topic3  -5.6533  -0.1796

[238 rows x 6 columns], token_table=      Topic      Freq    Term
term                         
106       1  0.498784  Italia
106       2  0.107522  Italia
106       3  0.391261  Italia
443       1  0.005192    Roma
443       2  0.384180    Roma
...     ...       ...     ...
677       2  0.130258   volta
677       3  0.561457   volta
188       1  0.151628    voto
188       2  0.062206    voto
188       3  0.789243    voto

[357 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 2, 1])

### Topics:
- Topic 0: "Political Opposition and Governance Critique": 
    This topic focuses on political discourse and critique of right-wing governance. Key terms include "politica" (politics), "governo" (government), "destra" (right/right-wing), "Roma" (Rome), "politico" (political), "voto" (vote), and "partito" (party). This suggests rhetoric centered on political opposition, electoral strategy, and criticism of right-wing politics, with a focus on serious ("serio") political engagement.
- Topic 1: "Rights and Social Justice": 
    This topic emphasizes rights, freedoms, and social concerns. Prominent words include "diritto" (right/rights), "libertà" (freedom), "lavoro" (work), "donna" (woman), and "persona" (person). The presence of "senza" (without) suggests discourse around inequality and exclusion. This topic reflects left-wing emphasis on individual rights, gender equality, and social justice concerns.
- Topic 2: "Economic Policy and Labor Rights": 
    This topic centers on economic governance and labor issues. Key terms include "governo" (government), "lavoro" (work), "drago" (likely referring to Mario Draghi or economic policies), "paese" (country), "lavoratore" (worker), "crisi" (crisis), and "proposta" (proposal). This suggests rhetoric focused on labor rights, economic crisis management, legislative proposals, and parliamentary action to address workers' concerns.